In [1]:
import requests
from bs4 import BeautifulSoup
from pandas import DataFrame


def get_hospital(zone_url, hospital_dict):
    headers2 = {"User-Agent": "", "Cookie": ""}
    zone_content = requests.get(zone_url, headers=headers2)

    # rint(zone_content.text)
    zone_soup = BeautifulSoup(zone_content.text, "html.parser")
    zone_target = zone_soup.find_all("div", class_="listItem")
    # print(zone_target)

    for item in zone_target:
        name = item.a.get_text(strip=True)
        # print(name)
        a_label = item.find_all("a")[0]
        # print(a_label['href'])
        hospital_dict[name] = a_label['href']
        # print()
    next_url = None
    next_page = zone_soup.find("div", class_="endPage")
    if next_page:
        next_link = next_page.find("a", class_="next")
        if next_link:
            next_url = next_link["href"]

    return hospital_dict, next_url


headers = {"User-Agent":"","Cookie":""}
xian_url="https://yyk.familydoctor.com.cn/area_305_0_0_0_1.html"
content = requests.get(xian_url,headers=headers)
#print(content.text)

xian_soup = BeautifulSoup(content.text,"html.parser")
target = xian_soup.find("div",class_="selection")
area_list = target.find_all("li")[1:]
area_dict={}
for area in area_list:
    area_name = area.get_text(strip=True)
    area_dict[area_name]=area.a["href"]
print(area_dict)

hospitals = {}
for zone in area_dict:
    hospitals,next_page = get_hospital(area_dict[zone],hospitals)
    while next_page:
        hospitals,next_page = get_hospital(next_page,hospitals)
    print(f"{zone} 医院信息已获取完毕...")

print(hospitals)

{'莲湖区': 'https://yyk.familydoctor.com.cn/addarea_2637_0_0_0_1.html', '新城区': 'https://yyk.familydoctor.com.cn/addarea_2652_0_0_0_1.html', '碑林区': 'https://yyk.familydoctor.com.cn/addarea_2660_0_0_0_1.html', '灞桥区': 'https://yyk.familydoctor.com.cn/addarea_2673_0_0_0_1.html', '未央区': 'https://yyk.familydoctor.com.cn/addarea_2681_0_0_0_1.html', '雁塔区': 'https://yyk.familydoctor.com.cn/addarea_2697_0_0_0_1.html', '阎良区': 'https://yyk.familydoctor.com.cn/addarea_2720_0_0_0_1.html', '临潼区': 'https://yyk.familydoctor.com.cn/addarea_2732_0_0_0_1.html', '长安区': 'https://yyk.familydoctor.com.cn/addarea_2742_0_0_0_1.html', '蓝田县': 'https://yyk.familydoctor.com.cn/addarea_2749_0_0_0_1.html', '周至县': 'https://yyk.familydoctor.com.cn/addarea_2754_0_0_0_1.html', '户县': 'https://yyk.familydoctor.com.cn/addarea_2758_0_0_0_1.html', '高陵县': 'https://yyk.familydoctor.com.cn/addarea_2761_0_0_0_1.html'}
莲湖区 医院信息已获取完毕...
新城区 医院信息已获取完毕...
碑林区 医院信息已获取完毕...
灞桥区 医院信息已获取完毕...
未央区 医院信息已获取完毕...
雁塔区 医院信息已获取完毕...
阎良区 医院信息已获取完毕.

In [2]:
print(hospitals)

{'西安市红会医院': 'https://yyk.familydoctor.com.cn/17097/', '陕西省中医医院': 'https://yyk.familydoctor.com.cn/16920/', '西安市第五医院': 'https://yyk.familydoctor.com.cn/16808/', '西安莲湖华中医院': 'https://yyk.familydoctor.com.cn/17002/', '陕西西京中医医院': 'https://yyk.familydoctor.com.cn/16999/', '西安莲湖中童儿童康复医院': 'https://yyk.familydoctor.com.cn/16929/', '陕西省皮肤性病防治所': 'https://yyk.familydoctor.com.cn/16909/', '西安北方中医皮肤病医院': 'https://yyk.familydoctor.com.cn/20508/', '西安莲湖长城医院': 'https://yyk.familydoctor.com.cn/21384/', '西安中际中西医结合脑病医院': 'https://yyk.familydoctor.com.cn/20871/', '西安莲湖大唐医院(男科)': 'https://yyk.familydoctor.com.cn/20909/', '西安莲湖国豪医院': 'https://yyk.familydoctor.com.cn/20534/', '西安莲湖生殖医院': 'https://yyk.familydoctor.com.cn/20994/', '西安市莲湖区红十字会医院': 'https://yyk.familydoctor.com.cn/16933/', '西安市莲湖区妇幼保健站': 'https://yyk.familydoctor.com.cn/16959/', '西安市莲湖区中医院': 'https://yyk.familydoctor.com.cn/16968/', '西安市立山医院': 'https://yyk.familydoctor.com.cn/16973/', '西安市莲湖区庙后街医院': 'https://yyk.familydoctor.com.cn/16993/', '西

In [3]:
excel_dict={}
df = DataFrame(columns=["医院名称","医院类型","医院等级","医院地址","咨询电话"])
for hospital in hospitals:
    print("医院名称：",hospital)
    hospital_link=hospitals[hospital]
    headers = {"User-Agent":"","Cookie":""}
    hospital_info = requests.get(hospital_link,headers=headers)

    hospital_soup = BeautifulSoup(hospital_info.text,"html.parser")
    temp = hospital_soup.find_all("div",class_="introPic")
    if temp:
        hospital_intro = temp[0]
        #print(hospital_intro)
        detail = hospital_intro.find_all("dl")
        #print(detail)
        temp_dict={}
        for subitem in detail:
            title = subitem.dt.get_text(strip=True)
            if "医院地址" in title:
                answer = subitem.dd.get_text(strip=True)[:-4]
            else:
                answer = subitem.dd.get_text(strip=True)
            temp_dict[title]=answer
            print(title,answer)
    excel_dict[hospital]=temp_dict
    print(temp_dict)
    df=df.append({"医院名称":hospital,"医院类型":temp_dict["医院类型："],"医院等级":temp_dict["医院等级："],"医院地址":temp_dict["医院地址："],"咨询电话":temp_dict["咨询电话："]},ignore_index=True)

    print("======================")
    print(df)
    
df.to_excel("result.xls")
print("结果写入 result.xls")

医院名称： 西安市红会医院
医院类型： 专科医院
医院等级： 三级甲等
医院地址： 中国陕西省西安市南稍门南郭路76号
咨询电话： 门诊咨询（8:00-18:00）18802940114 ,医疗总值班（18:00-8:00）15129265553,行政总值班13892893540（24小时）
{'医院类型：': '专科医院', '医院等级：': '三级甲等', '医院地址：': '中国陕西省西安市南稍门南郭路76号', '咨询电话：': '门诊咨询（8:00-18:00）18802940114 ,医疗总值班（18:00-8:00）15129265553,行政总值班13892893540（24小时）'}
      医院名称  医院类型  医院等级               医院地址  \
0  西安市红会医院  专科医院  三级甲等  中国陕西省西安市南稍门南郭路76号   

                                                咨询电话  
0  门诊咨询（8:00-18:00）18802940114 ,医疗总值班（18:00-8:00）...  
结果写入 result.xls
